In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import json
import codecs
import jieba
import numpy
import math

In [43]:
def news_vector_dict(file_root, title_scale, doc_scale, min_df, max_df):
    file = codecs.open(file_root, 'r', 'utf-8')
    news_dict = json.load(file)
    
    # 分词，在词之间加空格，重新组成文章
    stop_file = codecs.open('./data/stop_words.txt', 'r', 'utf-8')
    stop_list = stop_file.read().split('\n')
    i = 0
    title_array = []
    doc_array = []
    for news_key in news_dict:
        title_text = news_dict[news_key][0]
        _title = jieba.lcut(title_text)
        for w in _title[:]:
            if w.split('.')[0].isdigit():
                _title.remove(w)
        title = ' '.join(_title)
        
        doc_text = news_dict[news_key][1]
        _doc = jieba.lcut(doc_text)
        for w in _doc[:]:
            if w.split('.')[0].isdigit():
                _doc.remove(w)
        doc = ' '.join(_doc)
        title_array.append(title)
        doc_array.append(doc)
        i += 1
        if i % 1000 == 0:
            print(i)
    
    # tf-idf算法，文章转化为一个归一化的向量
#     tfidf_vectorizer = TfidfVectorizer(min_df = min_df, stop_words = stop_list)
    tfidf_vectorizer = TfidfVectorizer(min_df = min_df, max_df = max_df, stop_words = stop_list)
    tfidf_vectorizer.fit(doc_array)
    doc_matrix = tfidf_vectorizer.transform(doc_array)
#     tfidf_vectorizer = TfidfVectorizer(stop_words = stop_list)
#     tfidf_vectorizer.fit(doc_array)
    title_matrix = tfidf_vectorizer.transform(title_array)
    
    word_bag = {}
    for key in tfidf_vectorizer.vocabulary_:
        word_bag.setdefault(tfidf_vectorizer.vocabulary_[key], key)
    
    # 计算文章加权vector
    news_matrix = (title_matrix.todense() * title_scale + doc_matrix.todense() * doc_scale).tolist()
#     _t = title_matrix.todense().tolist()
#     _d = doc_matrix.todense().tolist()
    
    # 构建news_key : vector字典
    i = 0
    news_vector_dict = {}
    for news_key in news_dict:
        news_vector_dict.setdefault(news_key, news_matrix[i])
        if i % 1000 == 0:
            print('i='+str(i))
#             print(news_matrix[i][:10])
            
         #打印文章关键词和权重
#         if i < 15:
#             news_words = []
#             news_words_weight = []
#             for j in range(len(news_matrix[i])):
#                 if news_matrix[i][j] > 0:
#                     news_words.append(word_bag[j])
#                     news_words_weight.append(news_matrix[i][j])
#             print(news_words)
#             print(news_words_weight)

#             _t_words = []
#             _t_words_weight = []
#             for j in range(len(_t[i])):
#                 if _t[i][j] > 0:
#                     _t_words.append(word_bag[j])
#                     _t_words_weight.append(_t[i][j])
#             print(_t_words)
#             print(_t_words_weight)
#             print(title_array[i])
#             print(jieba.lcut(news_dict[news_key][0]))

#             _d_words = []
#             _d_words_weight = []
#             for j in range(len(_d[i])):
#                 if _d[i][j] > 0:
#                     _d_words.append(word_bag[j])
#                     _d_words_weight.append(_d[i][j])
#             print(_d_words)
#             print(_d_words_weight)
#             print(doc_array[i])
#             print(jieba.lcut(news_dict[news_key][1]))

            print('-------------------------')
    
        i += 1
                    

    
    return news_vector_dict

file_root = './data/_news_data_clean.json'
title_scale = 0.6
doc_scale = 1.0 - title_scale
min_df = 5
max_df = 50
_news_vector_dict = news_vector_dict(file_root, title_scale, doc_scale, min_df, max_df)

1000
2000
3000
i=0
-------------------------
i=1000
-------------------------
i=2000
-------------------------
i=3000
-------------------------


In [44]:
def time_back(t):
    a = int(t-1393603200)
    return int(a / 86400)

def user_vector_dict(news_vector_dict):
    file = codecs.open('./data/_user_data_training_clean.json', 'r', 'utf-8')
    news_data = codecs.open('./data/_news_data_clean.json', 'r', 'utf-8')
    user_dict = json.load(file)
    news_d = json.load(news_data)
    
    j = 0
    user_vector_dict = {}
    # 每一个用户
    for user_key in user_dict:
        # 该用户读过的所有新闻的向量和为用户向量
        i = 0
        time_scale = 0
        vector_sum = numpy.matrix('0.0')
        for user_news_key in user_dict[user_key]:
            vector = numpy.matrix(news_vector_dict[user_news_key])
            time_scale = time_back(news_d[user_news_key][2]) / 5 + 1
            vector_sum = vector * 1 + vector_sum
            i += 1
        if i != 0:
            vector_sum /= i
        user_vector_dict.setdefault(user_key, vector_sum.tolist()[0])
        j += 1
        if j % 1000 == 0:
            print('j='+str(j))
            print(vector_sum.tolist()[0][:10])
    return user_vector_dict


_user_vector_dict = user_vector_dict(_news_vector_dict)

In [52]:
def k_n_n(news_dict, user_dict, k):
    news_keys = []
    news = []
    i = 0
    for news_key in news_dict:
        news_keys.append(news_key)
        news.append(news_dict[news_key])
        i += 1
        if i % 1000 == 0:
            print(i)
    
    print("training...")
    neigh = NearestNeighbors(n_neighbors=k)
    neigh.fit(news)
    
    user_keys = []
    nbrs = []
    i = 0
    for user_key in user_dict:
        users = []
        user_keys.append(user_key)
        users.append(user_dict[user_key])
        nbrs += neigh.kneighbors(users)
        i += 1
        if i % 50 == 0:
            print(i)
    
#     n = neigh.kneighbors(users)
    print(nbrs[:10])
    return nbrs

k = 100
n = k_n_n(_news_vector_dict, _user_vector_dict, k)

1000
2000
3000
training...
50
100
150
200
250
300
[array([[ 0.13725399,  0.13725399,  0.13725399,  0.13725399,  0.40412154,
         0.40483247,  0.40565588,  0.40820153,  0.41137378,  0.41206212,
         0.41289344,  0.41291536,  0.41325662,  0.41353262,  0.41440022,
         0.41462821,  0.41538306,  0.41558451,  0.41581106,  0.41583959,
         0.41628558,  0.41628558,  0.41632389,  0.41647987,  0.41651035,
         0.41682134,  0.41692377,  0.41701258,  0.41705184,  0.41708361,
         0.41714778,  0.41729258,  0.41742647,  0.41744574,  0.41749359,
         0.41759149,  0.41770426,  0.41781692,  0.41789309,  0.41801685,
         0.41807082,  0.41809375,  0.41811316,  0.41836768,  0.41852713,
         0.41868848,  0.41868931,  0.4187162 ,  0.41878029,  0.41882678,
         0.41890447,  0.41893123,  0.418945  ,  0.41913487,  0.41920047,
         0.41933147,  0.41933327,  0.41933597,  0.41935547,  0.41936941,
         0.41947376,  0.41955416,  0.41958004,  0.41961219,  0.41964065,


In [53]:
def time_scale(t):
    day=int((t-1393603200)/86400)
    if day < 1:
        day = 1
    return (math.log(day)+1)

news_keys = []
i = 0
for news_key in _news_vector_dict:
    news_keys.append(news_key)

    
file = codecs.open('./data/_user_data_training_clean.json', 'r', 'utf-8')
f_news_data = codecs.open('./data/_news_data_clean.json', 'r', 'utf-8')
user_news_dict = json.load(file)
news_data = json.load(f_news_data)

result = {}
i = 0
lens = []
pair = []
for user_key in user_news_dict:
    dist = n[2*i][0].tolist()
    indices = n[2*i+1][0].tolist()
    pair = []
    for m in range(len(indices)):
        mth = indices[m]
        news_id = news_keys[mth]
        if news_id in user_news_dict[user_key]:
            continue
    
        time_ratio = time_scale(news_data[news_id][2])
        #print(time_ratio, dist[m], dist[m]*time_ratio)
        pair.append([dist[m] * time_ratio, news_id])
    
    pair.sort(key=lambda x:x[0],reverse=True)
    user_news_keys = []
    for k in range(30):
        user_news_keys.append(pair[k][1])
    result.setdefault(user_key, user_news_keys)
    if i < 10:
        for p in pair:
            print(p[0], time_scale(news_data[p[1]][2]))
        print("\n")
    
#         print(len(result[user_key]))
#         print(result[user_key])
#         print(user_news_dict[user_key])
    i += 1
    lens.append(len(result[user_key]))
lens.sort()
print(lens[:10])

# news_keys = []
# i = 0
# for news_key in _news_vector_dict:
#     news_keys.append(news_key)

    
# file = codecs.open('./data/_user_data_training_clean.json', 'r', 'utf-8')
# user_news_dict = json.load(file)

# result = {}
# i = 0
# lens = []
# for user_key in user_news_dict:
#     indices = n[2*i+1][0].tolist()
#     user_news_keys = []
#     for index in indices:
#         user_news_key = news_keys[index]
#         if user_news_key not in user_news_dict[user_key]:
#             user_news_keys.append(user_news_key)
#     result.setdefault(user_key, user_news_keys)
# #     if i < 100:
# #         print(len(result[user_key]))
# #         print(result[user_key])
# #         print(user_news_dict[user_key])
#     i += 1
#     lens.append(len(result[user_key]))
# lens.sort()
# print(lens[:10])



[59, 61, 62, 69, 70, 72, 77, 77, 79, 80]


In [54]:
# print(1)
# print(n[:6])
file_output = codecs.open('./data/tfidf_result.json', 'w', 'utf-8')
json.dump(result, file_output)
file_output.close()